<a href="https://colab.research.google.com/github/neohack22/ebw3nt/blob/main/Correction_de_Cross_validation_evaluating_estimator_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

1. **`from sklearn import svm`**

   * Tu importes le module `svm` de scikit-learn, qui permet de créer des **Support Vector Machines** (SVM).
   * Les SVM sont des modèles de classification (et régression) qui cherchent à séparer les classes dans l’espace des features par des hyperplans optimaux.

2. **`X, y = datasets.load_iris(return_X_y=True)`**

   * `datasets.load_iris()` charge le célèbre **jeu de données Iris**, qui contient 150 échantillons de fleurs répartis en 3 espèces.
   * `X` contient les **features** (longueur et largeur des sépales et pétales), donc une matrice de 150×4.
   * `y` contient les **labels**, c’est-à-dire l’espèce de chaque fleur (0, 1 ou 2).
   * L’option `return_X_y=True` renvoie directement les deux tableaux séparés, au lieu d’un objet plus complexe.

> 1. Séparer les données en **jeu d’entraînement** et **jeu de test**, en réservant **40 % des données pour le test**.
> 2. Fixer un **random_state = 0** pour que les résultats soient reproductibles.
> 3. Afficher la **taille** des tableaux d’entraînement `X_train` et `y_train`.

**Questions guidées :**

* Quelle fonction de `sklearn` permet de diviser les données de cette manière ?
* Comment vérifier rapidement les dimensions des arrays obtenus ?

**Bonus challenge :**

* Essayez avec un test_size différent (par exemple 0.3) et observez ce qui change.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

X_train.shape, y_train.shape

((90, 4), (90,))

In [ ]:
X_test.shape, y_test.shape

((60, 4), (60,))

In [ ]:
# clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clm = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clm.score(X_test, y_test)

0.9666666666666667

Un **SVM** cherche à séparer des données en **deux classes**. L’**hyperplan** est simplement la "ligne" (en 2D) ou le "plan" (en 3D) qui sépare ces deux classes.

* En 2 dimensions, c’est une droite.
* En 3 dimensions, c’est un plan.
* Dans plus de 3 dimensions, c’est un **hyperplan**, mais le concept est le même : il sépare l’espace en deux parties, chacune correspondant à une classe.

Le SVM cherche **l’hyperplan qui maximise la marge**, c’est-à-dire la distance entre les points les plus proches de chaque classe (appelés **vecteurs de support**) et l’hyperplan.

---

Le paramètre `C` contrôle **l’équilibre entre la précision sur les données d’entraînement et la simplicité de l’hyperplan** :

* **C grand (ex. 1000)** : le modèle essaie de **corriger toutes les erreurs d’entraînement**, même si l’hyperplan devient très complexe.
* **C petit (ex. 0.1)** : le modèle tolère **quelques erreurs sur l’entraînement**, pour avoir un hyperplan plus simple et qui **généralise mieux** sur de nouvelles données.

Image :

* Grand C → tu colles l’hyperplan à chaque point (risque de surapprentissage).
* Petit C → tu laisses quelques points "mal classés" mais le modèle est plus robuste.

\### Calcul de métriques validées par validation croisée

1. Créer un **classifieur SVM** avec un **kernel linéaire** et un paramètre `C=1`.
2. Évaluer la performance de votre modèle à l’aide de la **validation croisée à 5 plis**.
3. Afficher les **scores de chaque pli** pour analyser la stabilité de votre modèle.

> Indice : Utilisez `sklearn.model_selection.cross_val_score` et `sklearn.svm.SVC`.

**Objectifs pédagogiques :**

* Comprendre comment utiliser un SVM avec kernel linéaire.
* Appliquer la validation croisée pour estimer la performance d’un modèle.
* Interpréter les scores obtenus sur différents folds.

In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.98 accuracy with a standard deviation of 0.02


L’écart-type **mesure la dispersion des valeurs autour de la moyenne**.

* Si les valeurs sont très proches de la moyenne → écart-type petit.
* Si les valeurs sont très éparpillées → écart-type grand.

Supposons que tu as les scores d’un modèle sur 5 tests :

```
scores = [80, 82, 78, 79, 81]
```

* Moyenne : (80+82+78+79+81)/5 = 80
* Écart par rapport à la moyenne :

  * 80 → 0
  * 82 → +2
  * 78 → -2
  * 79 → -1
  * 81 → +1

L’écart-type combine ces écarts (en les mettant au carré, en faisant la moyenne, puis en prenant la racine) pour donner une **mesure unique de dispersion**. Ici, il serait petit (≈1.58) car les scores sont proches de 80.

* Pour un modèle de machine learning : si l’écart-type des scores de validation croisée est petit, ton modèle **est stable**.
* Si l’écart-type est grand, les performances varient beaucoup selon les sous-ensembles de données, et le modèle est **moins fiable**.

Évaluez la performance de votre modèle en utilisant une validation croisée 5 folds et le score F1 macro.
Affichez la liste des scores obtenus pour chaque fold.

In [ ]:
from sklearn import metrics
scores = cross_val_score(
    clf, X, y, cv=5, scoring='f1_macro')
scores

array([0.96658312, 1.        , 0.96658312, 0.96658312, 1.        ])

Votre objectif est de **tester la robustesse de votre modèle** en utilisant une technique de validation croisée qui mélange aléatoirement les échantillons à chaque itération.
>
> 1. Créez un objet de validation croisée qui divise vos données en 5 itérations, avec **30 % des données utilisées pour le test à chaque fois**. Assurez-vous que les tirages soient reproductibles.
> 2. Utilisez cet objet pour **évaluer la performance de votre classifieur** et afficher les scores pour chaque itération.
>
> *Indice : la classe `ShuffleSplit` de `sklearn.model_selection` peut vous être utile.*

In [ ]:
from sklearn.model_selection import ShuffleSplit
n_samples = X.shape[0]
# cv = ShuffleSplit(n_split=5, test_size=0.3, random_state=0)
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(clf, X, y, cv=cv)

array([0.97777778, 0.97777778, 1.        , 0.95555556, 1.        ])

> 1. Écrivez une fonction `custom_cv_2folds(X)` qui, à chaque itération, retourne les indices de train et test pour un pli.
> 2. Assurez-vous que votre fonction peut être utilisée directement avec `cross_val_score` de `scikit-learn`.
> 3. Testez votre fonction sur un classifieur `clf` et les données `X`, `y`.

**Points à réfléchir :**

* Comment diviser votre jeu de données en 2 moitiés égales pour les folds ?
* Quelle structure Python permet de renvoyer les indices **au fur et à mesure** pour chaque fold ?
* Pourquoi utiliser `np.arange` plutôt que `range` pour les indices ?

**Bonus challenge :**

* Modifiez votre fonction pour qu’elle accepte un nombre variable de folds (`n_folds`) et fonctionne pour n’importe quelle taille de dataset.


In [ ]:
def custom_cv_2folds(X):
  n = X.shape[0]
  i = 1
  while i<= 2:
    # idx = np.arrange(n * (i - 1) / 2, n * i / 2, dtype=int)
    idx = np.arange(n * (i - 1) / 2, n * i / 2, dtype=int)
    yield idx, idx
    i += 1

custom_cv = custom_cv_2folds(X)
cross_val_score(clf, X, y, cv=custom_cv)

array([1.        , 0.97333333])

1. Comment pouvez-vous séparer vos données en un jeu d’entraînement (60%) et un jeu de test (40%) tout en assurant la reproductibilité des résultats ?
2. Quel type de transformation appliqueriez-vous sur `X_train` pour que chaque feature ait une moyenne de 0 et un écart type de 1 ? Comment appliquer la même transformation sur `X_test` ?
3. En utilisant `sklearn`, quel modèle SVM pouvez-vous entraîner sur vos données normalisées ?
4. Comment mesurer la **précision** de votre modèle sur le jeu de test ?

In [ ]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0
)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)

0.9333333333333333

In [ ]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
cross_val_score(clf, X, y, cv=cv)

array([0.97777778, 0.93333333, 0.95555556, 0.93333333, 0.97777778])

La fonction `make_pipeline` permet de **chaîner plusieurs étapes de traitement** et un modèle final.
Chaque étape peut être, par exemple :

* une transformation des données (standardisation, encodage, etc.)
* un modèle de machine learning

Cela simplifie la préparation et l’entraînement, car tout se fait **dans un seul objet**.

---

`StandardScaler` est un **préprocesseur** qui :

* centre les données autour de 0
* les met à l’échelle avec un écart-type de 1

En gros, chaque caractéristique (feature) devient comparable, ce qui est important pour des modèles sensibles à l’échelle, comme le **SVM**.

#### La fonction cross_validate et l'évaluation de plusieurs métriques

1. Créer un classificateur SVM linéaire avec `C=1` et `random_state=0`.
2. Évaluer ce modèle en utilisant **la validation croisée** sur tout le jeu de données.
3. Calculer au moins deux métriques de performance pour chaque pli : **précision macro** et **rappel macro**.
4. Stocker les résultats dans un objet `scores` et afficher toutes les clés disponibles après l’évaluation.

**Questions-guides :**

* Quelle fonction de scikit-learn permet de faire une validation croisée et récupérer plusieurs métriques à la fois ?
* Comment spécifier les métriques que vous voulez calculer ?
* Comment instancier un SVM linéaire avec des paramètres spécifiques ?

**Bonus :** Essayez d’ajouter la métrique `f1_macro` et observez comment les résultats changent.


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [ ]:
scores['test_recall_macro']

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

`scores['test_recall_macro']` contient le **rappel moyen macro calculé sur l’ensemble de test** (ou via une validation croisée si tu utilises `cross_validate`).

In [ ]:
from sklearn.metrics import make_scorer
scoring = {'prec_macro': 'precision_macro',
           'rec_macro': make_scorer(recall_score, average='macro')}
scores = cross_validate(clf, X, y, scoring=scoring,
                        cv=5, return_train_score=True)
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_prec_macro',
 'test_rec_macro',
 'train_prec_macro',
 'train_rec_macro']

Ce code permet d’évaluer un modèle de classification sur plusieurs métriques (précision et rappel), de façon robuste grâce à la validation croisée. On peut comparer les performances sur l’entraînement et le test pour détecter un surapprentissage.

* `precision_macro` : la précision moyenne sur toutes les classes.
* `rec_macro` : le rappel moyen sur toutes les classes, créé avec `make_scorer` pour transformer la fonction `recall_score` en objet que `cross_validate` peut utiliser.

> On choisit `macro` pour traiter toutes les classes de façon **égale**, même si certaines sont minoritaires.

```python
sorted(scores.keys())
```
 liste toutes les clés disponibles dans le dictionnaire `scores`.
* Typiquement, on y trouve :

  * `'fit_time'` → temps pour entraîner le modèle
  * `'score_time'` → temps pour prédire
  * `'test_prec_macro'` → précision macro sur test
  * `'train_prec_macro'` → précision macro sur train
  * `'test_rec_macro'` → rappel macro sur test
  * `'train_rec_macro'` → rappel macro sur train

In [ ]:
scores['train_rec_macro']

array([0.975     , 0.975     , 0.99166667, 0.98333333, 0.98333333])

1. Utilisez la validation croisée à 5 folds pour calculer la **précision moyenne** (`precision_macro`) de votre modèle.
> 2. Faites en sorte de **récupérer les classifieurs entraînés à chaque pli**.
> 3. Inspectez la structure du résultat pour voir quelles informations sont disponibles (affichez les **clés** du dictionnaire retourné).
>
> **Indice** : pensez à une fonction de `sklearn.model_selection` qui permet de retourner à la fois les scores et les estimateurs.

In [ ]:
scores = cross_validate(clf, X, y,
                        scoring='precision_macro', cv=5,
                        return_estimator=True)
sorted(scores.keys())

['estimator', 'fit_time', 'score_time', 'test_score']

### Cross validation iterators

#### Cross-validation iterators for i.i.d. data

##### K-fold

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
  print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]


À chaque tour, on change la partie test pour valider la robustesse du modèle.

* `KFold` divise votre dataset en `n_splits` parties égales (ici 2).
* Chaque partie servira **une fois de test**, les autres parties serviront à l’entraînement.
* Exemple : si on a 6 observations et `n_splits=2` :

  * Split 1 : train = [0,1,2], test = [3,4,5]
  * Split 2 : train = [3,4,5], test = [0,1,2]

---

* `kf.split(X)` renvoie **les indices** des données pour l’entraînement (`train`) et le test (`test`) à chaque split.
* Dans la boucle, on peut :

  * **Entraîner le modèle** sur `X[train]`
  * **Tester le modèle** sur `X[test]`

In [ ]:
X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
#
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

##### Repeated K-Fold

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
random_state = 12883823
rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=random_state)
for train, test in rkf.split(X):
  print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]
[0 2] [1 3]
[1 3] [0 2]


`RepeatedKFold` est une technique de **validation croisée répétée**.

* `n_splits=2` : on divise les données en 2 groupes (folds).
* `n_repeats=2` : on répète ce processus **2 fois**, avec un nouveau découpage à chaque fois.
* `random_state=12883823` : assure que la partition est **répétable** (on obtient toujours les mêmes splits si on relance le code).

Chaque “split” donne :

* `train` : les indices des lignes utilisées pour l’entraînement.
* `test` : les indices des lignes utilisées pour le test.

---

* `rkf.split(X)` génère **tous les ensembles train/test** de toutes les répétitions.
* Le `print` affiche les indices des observations choisies pour l’entraînement et le test à chaque étape.

Pour notre exemple, on aura 4 affichages au total : 2 splits × 2 répétitions.

---

**Différence avec KFold simple :** ici, on répète la validation croisée pour réduire l’effet de la variance liée au découpage aléatoire.

##### Leave One Out (LOO)

In [ ]:
from sklearn.model_selection import LeaveOneOut

X = [1, 2, 3, 4]
loo = LeaveOneOut()
for train, test in loo.split(X):
  print("%s %s" % (train, test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


Leave-One-Out est une **méthode de validation croisée** utilisée pour évaluer la performance d’un modèle prédictif.
Le principe est simple :

* On prend **un échantillon de données** de taille (n).
* On entraîne le modèle sur **tous les points sauf un**.
* On teste le modèle sur **le point laissé de côté**.
* On répète cette opération **pour chaque point du jeu de données**.

En résumé : chaque observation est utilisée **exactement une fois pour tester** et **n-1 fois pour entraîner**.

---

* ✅ **Fiable pour les petits jeux de données**, car on maximise l’utilisation de chaque point (<100 observations).
* ✅ Permet d’obtenir une **estimation presque non biaisée** de la performance réelle du modèle.
* ⚠️ Cependant, c’est **très coûteux en calcul** si le dataset est grand.

---

Imaginez que vous formez un modèle pour prédire si un client va souscrire un produit financier (oui/non), avec 5 clients :

| Client | X (donnée) | Y (souscription) |
| ------ | ---------- | ---------------- |
| 1      | 10         | Oui              |
| 2      | 15         | Non              |
| 3      | 20         | Oui              |
| 4      | 25         | Non              |
| 5      | 30         | Oui              |

**Étapes LOO :**

1. On retire le client 1 du dataset.
2. On entraîne le modèle sur les clients 2 à 5.
3. On prédit la souscription du client 1 et on mesure l’erreur.
4. On répète pour chaque client.
5. On calcule **l’erreur moyenne** sur tous les clients.

##### Leave P Out (LPO)

In [ ]:
from sklearn.model_selection import LeavePOut

X = np.ones(4)
lpo = LeavePOut(p=2)
for train, test in lpo.split(X):
  print("%s %s" % (train, test))

[2 3] [0 1]
[1 3] [0 2]
[1 2] [0 3]
[0 3] [1 2]
[0 2] [1 3]
[0 1] [2 3]


Leave-P-Out est une méthode de **validation croisée**. Son objectif : tester la performance d’un modèle sur des données qu’il n’a **jamais vues** pendant l’entraînement.

* “P” signifie le **nombre d’exemples laissés de côté** à chaque itération.
* On entraîne le modèle sur toutes les données **sauf P**.
* On teste le modèle sur ces **P exemples laissés de côté**.
* On répète le processus **pour toutes les combinaisons possibles** de P exemples.

Supposons que vous ayez 5 observations (A, B, C, D, E) et que vous choisissiez **P = 2** :

* Première combinaison de test : (A, B), entraînement sur (C, D, E)
* Deuxième combinaison : (A, C), entraînement sur (B, D, E)
* … et ainsi de suite pour toutes les combinaisons de 2 observations.

À la fin, on fait la **moyenne des performances** sur toutes les combinaisons pour obtenir une estimation fiable de la qualité du modèle.

---

### Avantages

* Très **précis**, car le modèle est testé sur toutes les combinaisons possibles.
* Idéal pour des **petits jeux de données**.

---

### Inconvénients

* **Très coûteux en calcul** : le nombre de combinaisons est C(n, p)
* Pour de grands jeux de données, il devient **impraticable**.
* Rarement utilisé dans les grands projets : on préfère des alternatives comme K-Fold Cross-Validation.

---

### Quand l’utiliser

* **Petites datasets** où chaque observation compte.
* Quand on veut une **estimation très fiable** de la performance d’un modèle.
* Pour des études **académiques ou expérimentales**.

##### Validation croisée par permutations aléatoires (ou Shuffle & Split)

In [ ]:
from sklearn.model_selection import ShuffleSplit
X = np.arange(10)
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
for train_index, test_index in ss.split(X):
  print("%s %s" % (train_index, test_index))

[9 1 6 7 3 0 5] [2 8 4]
[2 9 8 0 6 7 4] [3 5 1]
[4 5 1 0 6 9 7] [2 3 8]
[2 7 5 8 0 3 4] [6 1 9]
[4 1 0 6 8 9 3] [5 2 7]


Au lieu de simplement couper vos données en **train** et **test** une seule fois, ShuffleSplit :

* Mélange (**shuffle**) les données de manière aléatoire.
* Les divise ensuite en **train/test** plusieurs fois.
* Cela permet d’avoir plusieurs évaluations du modèle sur différentes portions des données.

C’est particulièrement utile si :

* Vous avez peu de données.
* Vous voulez éviter que votre résultat dépende d’une seule séparation des données.

---

### Comment ça fonctionne concrètement ?

Imaginons un dataset de 1000 observations :

* Vous décidez que 80 % seront pour l’entrainement, 20 % pour le test.
* ShuffleSplit va **répéter ce découpage N fois** (par exemple 5 fois).
* À chaque répétition, les données sont **mélangées différemment**, donc le modèle est évalué sur des ensembles tests différents à chaque fois.

Résultat : vous obtenez une distribution de scores plus **fiable et représentative** des performances réelles.

---

### Pourquoi c’est intéressant pour des professionnels

* **Fiabilité** : évite de tirer des conclusions basées sur un seul découpage.
* **Flexibilité** : on peut ajuster la taille du test et le nombre de répétitions.
* **Simplicité** : facile à intégrer dans vos pipelines de validation.

#### Cross-validation iterators with stratification based on class labels.

##### Stratified k-fold

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
  print('train -  {}    | test  -   {}'.format(
      np.bincount(y[train]), np.bincount(y[test])))
#  ))

train -  [30  3]    | test  -   [15  2]
train -  [30  3]    | test  -   [15  2]
train -  [30  4]    | test  -   [15  1]


Ici, le problème est que les classes sont **déséquilibrées** : 0 est très majoritaire, 1 très minoritaire. Si on fait un simple découpage aléatoire, certains sous-ensembles pourraient **ne pas contenir de 1**, ce qui poserait problème pour l’évaluation.

---

`StratifiedKFold` est une variante de KFold qui **préserve la proportion de chaque classe dans chaque pli (fold)**.

* `n_splits=3` → on découpe les données en 3 sous-ensembles.
* `skf.split(X, y)` → retourne, pour chaque pli, les indices des données d’entraînement (`train`) et de test (`test`).

---

`np.bincount(y[train])` compte combien d’exemples de chaque classe sont dans le train (et pareil pour test).

Donc, on **voit la répartition des classes dans chaque fold**.

Par exemple, une sortie typique pourrait être :

```
train -  [30 3]    | test  -   [15 2]
train -  [30 3]    | test  -   [15 0]
train -  [31 2]    | test  -   [14 3]
```

* Les `0` et `1` sont **répartis de manière équilibrée selon leur proportion initiale**.
* On évite d’avoir un test set sans aucune instance minoritaire.

1. Diviser votre dataset en **3 folds** pour une validation croisée.
2. Pour chaque fold, récupérer les indices de l’ensemble d’entraînement et de test.
3. Afficher la répartition des classes (`y`) dans les ensembles d’entraînement et de test pour chaque fold.

**Indices :**

* Utilisez `KFold` de `sklearn.model_selection`.
* Les indices de train/test sont donnés par `kf.split(X, y)`.
* `np.bincount` peut vous aider à compter le nombre d’occurrences de chaque classe.

**Attendu (exemple de sortie) :**

```
train  -   [40 40]    |   test  -   [20 20]
train  -   [40 40]    |   test  -   [20 20]
train  -   [40 40]    |   test  -   [20 20]
```

In [ ]:
kf = KFold(n_splits=3)
for train, test in kf.split(X, y):
  print('train  -   {}    |   test  -   {}'.format(
      np.bincount(y[train]), np.bincount(y[test])
  ))

train  -   [28  5]    |   test  -   [17]
train  -   [28  5]    |   test  -   [17]
train  -   [34]    |   test  -   [11  5]


#### Itérateurs de validation croisée pour les données groupées.

##### K-fold de groupe

In [ ]:
from sklearn.model_selection import GroupKFold

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]

gkf = GroupKFold(n_splits=3)
for train, test in gkf.split(X, y, groups=groups):
  print("%s %s" % (train, test))

[0 1 2 3 4 5] [6 7 8 9]
[0 1 2 6 7 8 9] [3 4 5]
[3 4 5 6 7 8 9] [0 1 2]


`GroupKFold` est une **méthode de validation croisée** (cross-validation) utilisée quand on a des **groupes de données liés** et qu’on ne veut pas que des observations provenant du même groupe se retrouvent à la fois dans l’ensemble d’entraînement et dans l’ensemble de test.

* Exemple concret :

  * Vous entraînez un modèle sur des patients pour prédire une maladie.
  * Chaque patient peut avoir plusieurs observations (mesures à différents moments).
  * Si certaines mesures du même patient apparaissent à la fois dans l’entraînement et le test, le modèle "triche" car il voit des données très similaires.
  * **Solution : utiliser `GroupKFold`** pour que toutes les observations d’un patient soient dans **le même fold**, soit entraînement, soit test.

---

1. On fournit :

   * `X` : les données (features)
   * `y` : les labels (cibles)
   * `groups` : un vecteur qui indique à quel groupe appartient chaque observation

2. `GroupKFold` divise les données en `k` folds de manière à ce que **aucun groupe ne soit partagé entre les folds**.

3. Pour chaque itération :

   * Un fold devient le **test set**
   * Les autres folds deviennent le **train set**



**Les groupes ne sont jamais mélangés** entre train et test.

---

* Évite le **faux optimisme** du modèle.
* Garantit que le modèle est testé sur des **données vraiment inédites**, pas sur des variations très proches de celles vues à l’entraînement.
* Très utile dans :

  * Médecine (patients)
  * Marketing (clients)
  * Industrie (machines ou sites)

##### StratifiedGroupKFold

##### Leave One Group Out

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut

X = [1, 5, 10, 50, 60, 70, 80]
y = [0, 1, 1, 2, 2, 2, 2]
groups = [1, 1, 2, 2, 3, 3, 3]
logo = LeaveOneGroupOut()
for train, test in logo.split(X, y, groups=groups):
  print("%s %s" % (train, test))

[2 3 4 5 6] [0 1]
[0 1 4 5 6] [2 3]
[0 1 2 3] [4 5 6]


Au lieu de séparer les données aléatoirement, on les sépare selon des **groupes**.

* Chaque "groupe" peut être une catégorie, un patient, un client, une machine, un site, etc.
* L’idée : à chaque itération, **on laisse un groupe complet de côté pour tester**, et on entraîne le modèle sur tous les autres groupes.

> Exemple : si vous faites de la prédiction sur des patients, vous voulez tester votre modèle sur des patients **jamais vus** plutôt que de mélanger les données de chaque patient entre train et test.

* Évite le **faux sentiment de performance** que l’on obtient quand les données du même groupe se retrouvent à la fois dans l’entraînement et le test.
* Important quand les observations sont **corrélées au sein d’un groupe**.
* Donne une évaluation plus **réaliste** pour la généralisation sur de nouveaux groupes.

---

`groups` est un **vecteur (liste ou array)** qui associe **chaque observation à un groupe**.

* Même longueur que `X` et `y`.
* Chaque valeur indique à quel **groupe** appartient l’observation correspondante.

> Exemple : si vous avez 10 observations et 3 patients (A, B, C), `groups` pourrait ressembler à :

```python
groups = ['A','A','A','B','B','B','C','C','C','C']
```

Ici :

* Les 3 premières lignes → patient A
* Les 3 suivantes → patient B
* Les 4 dernières → patient C

Lors de la validation croisée **LeaveOneGroupOut** :

1. Le modèle va **laisser un groupe entier de côté** pour le test.
2. Toutes les observations avec le même identifiant de groupe sont **testées ensemble**.
3. Les autres groupes servent à **l’entraînement**.

> Avec l’exemple ci-dessus :

* Itération 1 → test = A, train = B+C
* Itération 2 → test = B, train = A+C
* Itération 3 → test = C, train = A+B

---

* Les groupes doivent être définis **avant la validation**, selon ce que vous voulez généraliser.
* Très utile pour éviter que le modèle “triche” en voyant des données similaires de test dans le train.
* Chaque groupe doit avoir **au moins une observation**, sinon LOGO ne pourra pas l’utiliser comme test.

* Chaque test est **entièrement indépendant des autres groupes**.
* Utile quand les groupes ont des caractéristiques internes similaires.
* Plus fiable que le simple K-Fold si vos données sont **corrélées par groupe**.
* Ne pas confondre avec le **LeaveOneOut classique** qui ignore les groupes.


##### Leave P Groups Out

In [ ]:
from sklearn.model_selection import LeavePGroupsOut

X = np.arange(6)
y = [1, 1, 1, 2, 2, 2]
groups = [1, 1, 2, 2, 3, 3]
lpgo = LeavePGroupsOut(n_groups=2)
for train, test in lpgo.split(X, y, groups=groups):
  print("%s %s" % (train, test))

[4 5] [0 1 2 3]
[2 3] [0 1 4 5]
[0 1] [2 3 4 5]


Imaginez que vous entraînez un modèle pour prédire les ventes dans différents magasins. Si votre modèle voit des données d’un magasin et qu’on le teste **sur ce même magasin**, il risque de “tricher” : il connaît déjà les habitudes locales.

**LeavePGroupsOut** règle ce problème :

* On **regroupe les données** (par exemple, chaque magasin = un groupe).
* À chaque test, on **laisse de côté un ou plusieurs groupes** et on entraîne le modèle sur tous les autres.
* On teste ensuite sur le ou les groupes laissés de côté, **jamais vus par le modèle**.

✅ Vous savez exactement si votre modèle peut **généraliser à de nouveaux magasins, clients ou patients**.

C’est comme étudier pour un examen, mais **tester vos connaissances sur des chapitres jamais vus en classe**.

##### Group Shuffle Split

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 0.001]
y = ["a", "b", "b", "b", "c", "c", "c", "a"]
groups = [1, 1, 2, 2, 3, 3, 4, 4]
gss = GroupShuffleSplit(n_splits=4, test_size=0.5, random_state=0)
for train, test in gss.split(X, y, groups=groups):
  print("%s %s" % (train, test))

[0 1 2 3] [4 5 6 7]
[2 3 6 7] [0 1 4 5]
[2 3 4 5] [0 1 6 7]
[4 5 6 7] [0 1 2 3]


**Intention :** éviter que les mêmes groupes apparaissent à la fois dans le train et dans le test.
**Pourquoi c’est crucial ?** Parce que sinon on teste le modèle sur des données *qu’il a déjà vues sous une autre forme*, ce qui gonfle artificiellement les performances.

Si un groupe va dans *train*, **l’ensemble de ses observations y vont**.
Si un groupe va dans *test*, **aucune de ses observations n’est dans le train**.

---

1. Tu définis un vecteur `groups`
   (ex : `groups = patient_id`, ou `machine_id`…).
2. GroupShuffleSplit :

   * mélange les groupes (pas les échantillons),
   * sélectionne un pourcentage de groupes pour le train,
   * le reste pour le test.

Cela garantit **zéro contamination** entre train et test.

---

Supposons 10 machines → chacune produit 500 lignes de données.
Tu veux que ton modèle détecte des anomalies.

Avec un train/test split classique :

* une même machine peut apparaître dans train **et** test → biais.

Avec **GroupShuffleSplit** :

* 8 machines dans train,
* 2 machines dans test,
* les 500 lignes de chaque machine restent ensemble.

Résultat :
**tu évalues la généralisation sur de vraies nouvelles machines**.

Situations typiques :

* données hiérarchiques ou corrélées,
* plusieurs observations par entité,
* temporel *sans* vouloir de split chronologique,
* tout contexte où les échantillons au sein d’un groupe se ressemblent trop.

#### Utilisation d'itérateurs de validation croisée pour diviser les ensembles d'entraînement et de test

In [ ]:
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

X = np.array([0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 0.001])
y = np.array(["a", "b", "b", "b", "c", "c", "c", "a"])
groups = np.array([1, 1, 2, 2, 3, 3, 4, 4])
train_indx, test_indx = next(
    GroupShuffleSplit(random_state=7).split(X, y, groups)
)
X_train, X_test, y_train, y_test = \
X[train_indx], X[test_indx], y[train_indx], y[test_indx]
X_train.shape, X_test.shape

((6,), (2,))

* `.split()` **ne renvoie pas directement les indices de lignes dans l’ordre**, ni “le premier” ou “le dernier” au sens mathématique, **il génère des paires d’indices (train, test) pour chaque division possible**.

  * Chaque paire contient **tous les indices des lignes correspondantes**, pas juste un indice unique.

* Exemple simple : si X a 5 lignes `[0,1,2,3,4]` et on fait 1 split :

  ```python
  gss = GroupShuffleSplit(n_splits=1, test_size=0.4, random_state=7)
  it = gss.split(X, y, groups)
  ```

  * `it` est un générateur qui **produira une seule paire** de listes d’indices :

    ```python
    train_index = [0,2,4]
    test_index  = [1,3]
    ```

> `next(it)` ne renvoie pas un seul “indice”, il renvoie **la première paire complète** d’indices (donc tous les indices pour l’entraînement et le test).

---

  * `next()` prend **la première sortie du générateur**, c’est-à-dire la **première paire train/test**.
  * Mais cette “première sortie” contient **tous les indices pour chaque ensemble**, donc ce n’est pas juste un nombre ou un indice unique.

* Si tu avais plusieurs splits (`n_splits=5`), chaque `next()` suivant te donnerait **la paire suivante** de train/test.

---

* Imagine un tiroir avec plusieurs **pochettes de cartes** : chaque pochette contient **toutes les cartes d’un jeu**.
* `.split()` prépare toutes les pochettes.
* `next()` prend la **première pochette entière**.
* Les cartes à l’intérieur = tous les indices pour entraîner et tester.

In [ ]:
np.unique(groups[train_indx]), np.unique(groups[test_indx])

(array([1, 2, 4]), array([3]))

##### Time Series Split

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)

TimeSeriesSplit(max_train_size=None, n_splits=3)


In [ ]:
for train, test in tscv.split(X):
  print("%s %s" % (train, test))

[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]


Avec des données **chronologiques** (ventes, températures, capteurs, stock, etc.), il y a une contrainte : **on ne peut pas mélanger le passé et le futur**.

* Exemple : tu ne peux pas utiliser les ventes de décembre pour prédire celles de janvier si ton modèle est censé être appliqué en temps réel.

C’est là que **TimeSeriesSplit** devient utile.

---

### Qu’est-ce que TimeSeriesSplit ?

C’est une **méthode de validation croisée adaptée aux séries temporelles**.
Contrairement à un KFold classique qui mélange aléatoirement les données :

* TimeSeriesSplit respecte **l’ordre chronologique**.
* Chaque "split" utilise **le passé pour prédire le futur**.

---

### Comment ça fonctionne concrètement ?

Supposons qu’on ait 9 observations et qu’on fasse `n_splits=3`. TimeSeriesSplit va créer 3 sous-ensembles d’entraînement et de test comme ceci :

| Split | Train         | Test |
| ----- | ------------- | ---- |
| 1     | 0 1 2         | 3 4  |
| 2     | 0 1 2 3 4     | 5 6  |
| 3     | 0 1 2 3 4 5 6 | 7 8  |

On voit que :

* Le **train** inclut toujours les données antérieures au **test**.
* Le **test** avance progressivement dans le temps.

---

### Exemple en Python

```python
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

X = np.arange(10).reshape(-1,1)  # features fictives
y = np.arange(10)  # target fictive

tscv = TimeSeriesSplit(n_splits=3)

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
```

Sortie possible :

```
TRAIN: [0 1 2 3] TEST: [4 5]
TRAIN: [0 1 2 3 4 5] TEST: [6 7]
TRAIN: [0 1 2 3 4 5 6 7] TEST: [8 9]
```

---

### Pourquoi c’est utile pour vous

* Permet de **tester des modèles prédictifs sur des séries temporelles** sans tricher avec le futur.
* Aide à **évaluer la stabilité** du modèle au fil du temps.
* S’intègre facilement dans **scikit-learn**, donc prêt pour les pipelines professionnels.

[Source : Scikit Learn](https://https://scikit-learn.org/stable/modules/cross_validation.html)